In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# pd.set_option('display.max_columns', None)
import scipy
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from  sklearn.metrics  import accuracy_score
pd.options.plotting.backend = "plotly"

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Tratamento de Dados
---

Funções do sklearn pra tratamento de texto
TfidVectorizer - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html?highlight=vectorizer#sklearn.feature_extraction.text.TfidfVectorizer

Esse tmb é de categorização, não da pra usar LabelEncoder - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder

Não da pra usar o OneHotEncoder no nosso problema, já que ele é mais focado pra categorizar labels, não frases
OneHotEncoder - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder


In [ ]:
#substituir todos 1 por 0 em label
labels = {0:"entailed",
          1:"neutral",
          2:"contradiction"}
train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")
train.label = train.label.replace(0,1)
print(train.label.unique())
train_en = train[train["lang_abv"]=="en"]

test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")
test_en = test[test["lang_abv"]=="en"]

# print("TRAIN")
# print(train.iloc[0])
# print("\n\n")
# print("TEST")
# print(test.iloc[0])


premise = train_en.premise
hypothesis = train_en.hypothesis

premise_train = train_en.premise[:5000]
hypothesis_train = train_en.hypothesis[:5000]

premise_test = train_en.premise[5000:]
# print(len(premise_test))
hypothesis_test = train_en.hypothesis[5000:]
# print(len(hypothesis_test))
tfidv = TfidfVectorizer()
tokenizer = tfidv.build_tokenizer()

In [ ]:
# labelCount = train_en.label.value_counts()
# labelCount.plot.barh()

In [ ]:
from sklearn import decomposition

x_train = tfidv.fit_transform(premise_train+" "+hypothesis_train)
x_test = tfidv.transform(premise_test+" "+hypothesis_test)

y_train = train_en.label[:5000].values
y_test = train_en.label[5000:].values

#testei algumas max_depth e outros argumentos opcionais mas nao mudou mto a precisao
clf = RandomForestClassifier(max_depth=2)
clf.fit(x_train, y_train)

predicted = clf.predict(x_test)
acc = accuracy_score(y_test,predicted)*100
print("RandomForest: %.1f"%acc+"%")

In [ ]:
from sklearn import decomposition
from sklearn.preprocessing import Normalizer

x_train = tfidv.fit_transform(premise_train+" "+hypothesis_train)
x_test = tfidv.transform(premise_test+" "+hypothesis_test)

#utilizar um PCA aqui dps do tfid
transformer = Normalizer().fit(x_train)
x_train = transformer.transform(x_train)
print("Normalizacao finalizada...")
pca = decomposition.PCA(n_components = 1000)
pca.fit(x_train.todense())
x_train = pca.transform(x_train.todense())
x_test = pca.transform(x_test.todense())
print("PCA finalizado...")
y_train = train_en.label[:5000].values
y_test = train_en.label[5000:].values

#pegar um modelo melhor
clf = RandomForestClassifier(max_depth=2)
clf.fit(x_train, y_train)
# print(clf.score(x_test,y_test))
predicted = clf.predict(x_test)
acc = accuracy_score(y_test,predicted)*100
print("RandomForest: %.1f"%acc+"%")

In [ ]:
x_train = tfidv.fit_transform(premise_train+" "+hypothesis_train)
x_test = tfidv.transform(premise_test+" "+hypothesis_test)

#utilizar um PCA aqui dps do tfid

y_train = train_en.label[:5000].values
y_test = train_en.label[5000:].values

#pegar um modelo melhor
clf = MultinomialNB()
clf.fit(x_train, y_train)
# print(clf.score(x_test,y_test))
predicted = clf.predict(x_test)
acc = accuracy_score(y_test,predicted)*100
print("MultinomialNB: %.1f"%acc+"%")

In [ ]:
from sklearn.linear_model import SGDClassifier
x_train = tfidv.fit_transform(premise_train+" "+hypothesis_train)
x_test = tfidv.transform(premise_test+" "+hypothesis_test)


#utilizar um PCA aqui dps do tfid
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(x_train)
x_train = transformer.transform(x_train)
print("normalizer finalizado...")

pca = decomposition.PCA(n_components = 1000)
pca.fit(x_train.todense())
x_train = pca.transform(x_train.todense())
x_test = pca.transform(x_test.todense())
print("PCA finalizado...")

y_train = train_en.label[:5000].values
y_test = train_en.label[5000:].values

#pegar um modelo melhor
clf = SGDClassifier()
clf.fit(x_train, y_train)
print("Treinamento finalizado...")
# print(clf.score(x_test,y_test))
predicted = clf.predict(x_test)
acc1 = accuracy_score(y_test,predicted)*100
print("SGDClassifier: %.1f"%acc1+"%")

In [ ]:
# print("frase tokenizada: \n",tokenizer(train_en.premise.iloc[0]),"\n")
# x_train = premise+"[SEP]"+hypothesis
# print(len(x_train))
# x_train = x_train.apply(tokenizer)
# print("Premissa e hipotese com separador e tokenizado:\n",x_train.iloc[0])
# # nao consegui dar fit no modelo com os dados tokenizados

In [ ]:
#tentei esse aqui mas ele da esse erro:
# TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.
# deixei essa operacao .toarray() por 10 minutos e nao rodou, entao desisti
# sPCA = decomposition.SparsePCA()
# sPCA.fit(x_train)
# x_train = sPCA.transform(x_train)

#TypeError: PCA does not support sparse input. See TruncatedSVD for a possible alternative.
# pca = decomposition.PCA()
# pca.fit(x_train)
# x_train = pca.transform(x_train)

# svd = decomposition.TruncatedSVD()
# svd.fit(x_train)
# x_train = svd.transform(x_train)